## NIREN PATEL IRDM STANCE DETECTION BOOK 2

In [0]:
#!kill -9 -1 # Kill Switch Do not run this cell

In [0]:
!ls

In [0]:
# Code to read csv file into colaboratory:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#2. Get the file
TrainProcessed = drive.CreateFile({'id':'12IgQVJPo-oznDjoy4qp-yBEv0wvjx-6I'}) 
TrainProcessed.GetContentFile('TrainProcessed.csv')  

ValidationProcessed = drive.CreateFile({'id':'1ao3ZMAwle7W9ZBPZ-lVPHmqFlhMHJ9hs'}) 
ValidationProcessed.GetContentFile('ValidationProcessed.csv')

TestProcessed = drive.CreateFile({'id':'1sOMZL3UQbnJXOxrDoQDBpm4a8GNeACaH'}) 
TestProcessed.GetContentFile('TestProcessed.csv')

In [2]:
#IMPORT PACKAGES
import numpy as np
import pandas as pd
import re
import math
from collections import Counter
import nltk
from nltk import word_tokenize
from nltk.tokenize import TweetTokenizer
nltk.download('punkt')
import string
from  itertools import chain
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package punkt to /content/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
#Read file as panda dataframe
TrainProcessed = pd.read_csv('TrainProcessed.csv')
ValidationProcessed = pd.read_csv('ValidationProcessed.csv')
TestProcessed = pd.read_csv('TestProcessed.csv')

In [0]:
#PUNCTUATION REMOVAL
TrainProcessed['Headline']         =  TrainProcessed["Headline"].apply(lambda x: re.sub(r'[^\w\s]', '',x))
TrainProcessed['articleBody']      =  TrainProcessed["articleBody"].apply(lambda x: re.sub(r'[^\w\s]', '',x))
TestProcessed['Headline']          =  TestProcessed["Headline"].apply(lambda x: re.sub(r'[^\w\s]', '',x))
TestProcessed['articleBody']       =  TestProcessed["articleBody"].apply(lambda x: re.sub(r'[^\w\s]', '',x))
ValidationProcessed['Headline']    =  ValidationProcessed["Headline"].apply(lambda x: re.sub(r'[^\w\s]', '',x))
ValidationProcessed['articleBody'] =  ValidationProcessed["articleBody"].apply(lambda x: re.sub(r'[^\w\s]', '',x))

In [0]:
#TOKENISATION
TrainProcessed['Headline']         =  TrainProcessed["Headline"].apply(nltk.word_tokenize)
TrainProcessed['articleBody']      =  TrainProcessed["articleBody"].apply(nltk.word_tokenize)
TestProcessed['Headline']          =  TestProcessed["Headline"].apply(nltk.word_tokenize)
TestProcessed['articleBody']       =  TestProcessed["articleBody"].apply(nltk.word_tokenize)
ValidationProcessed['Headline']    =  ValidationProcessed["Headline"].apply(nltk.word_tokenize)
ValidationProcessed['articleBody'] =  ValidationProcessed["articleBody"].apply(nltk.word_tokenize)

# LAPLACE SMOOTHING

In [0]:
#Function for QueryLieklihood
def QueryLikelihoodLaplaceSmoothing(dataset):
  DictionaryList= []
  for index, row  in dataset.iterrows():
    DictionaryHeadline = {}
    UniqueHeadline = []
    [UniqueHeadline.append(i) for i in row.Headline if not UniqueHeadline.count(i)]  
    
    for headlineWord in UniqueHeadline:
      DictionaryHeadline[headlineWord]=1 
      for bodyWord in row.articleBody:
        DictionaryHeadline[bodyWord]=1

    for headlineWord in UniqueHeadline:
      if headlineWord in row.articleBody:
        countWord = row.articleBody.count(headlineWord)
        DictionaryHeadline[headlineWord] = DictionaryHeadline[headlineWord] + countWord
      else:
        pass
    TotalWords = sum(DictionaryHeadline.values())
    for key in DictionaryHeadline:
      DictionaryHeadline[key] /= TotalWords
    DictionaryList.append(DictionaryHeadline)
  return DictionaryList

In [0]:
#Function for DocumentLikelihood
def DocumentLikelihoodLaplaceSmoothing(dataset):
  DictionaryList= []
  for index, row  in dataset.iterrows():
    DictionaryBody = {}
    UniqueBody = []
    [UniqueBody.append(i) for i in row.articleBody if not UniqueBody.count(i)]  

    for bodyWord in UniqueBody:
      DictionaryBody[bodyWord]=1
      for headlineWord in row.Headline:
        DictionaryBody[headlineWord]=1 


    for bodyWord in UniqueBody:
      if bodyWord in row.Headline:
        countWord = row.Headline.count(bodyWord)
        DictionaryBody[bodyWord] = DictionaryBody[bodyWord] + countWord
      else:
        pass
    TotalWords = sum(DictionaryBody.values())
    for key in DictionaryBody:
      DictionaryBody[key] /= TotalWords
    DictionaryList.append(DictionaryBody)
  return DictionaryList

In [0]:
#Get Prbability Distributions for each model
TrainQueryLikelihoodLaplace         = QueryLikelihoodLaplaceSmoothing(TrainProcessed)
ValidationQueryLikelihoodLaplace    = QueryLikelihoodLaplaceSmoothing(ValidationProcessed)
TestQueryLikelihoodLaplace          = QueryLikelihoodLaplaceSmoothing(TestProcessed)

TrainDocumentLikelihoodLaplace      = DocumentLikelihoodLaplaceSmoothing(TrainProcessed)
ValidationDocumentLikelihoodLaplace = DocumentLikelihoodLaplaceSmoothing(ValidationProcessed)
TestDocumentLikelihoodLaplace       = DocumentLikelihoodLaplaceSmoothing(TestProcessed)


# LIDSTONE CORRECTION

In [0]:
#Function for QueryLieklihood
def QueryLikelihoodDiscounting(dataset):
  DictionaryList= []
  for index, row  in dataset.iterrows():
    DictionaryHeadline = {}
    UniqueHeadline = []
    [UniqueHeadline.append(i) for i in row.Headline if not UniqueHeadline.count(i)]  
    
    for headlineWord in UniqueHeadline:
      DictionaryHeadline[headlineWord]=0.2
      for bodyWord in row.articleBody:
        DictionaryHeadline[bodyWord]=0.2

    for headlineWord in UniqueHeadline:
      if headlineWord in row.articleBody:
        countWord = row.articleBody.count(headlineWord)
        DictionaryHeadline[headlineWord] = DictionaryHeadline[headlineWord] + countWord
      else:
        pass
    TotalWords = sum(DictionaryHeadline.values())
    for key in DictionaryHeadline:
      DictionaryHeadline[key] /= TotalWords
    DictionaryList.append(DictionaryHeadline)
  return DictionaryList

In [0]:
#Function for DocumentLikelihood
def DocumentLikelihoodDiscounting(dataset):
  DictionaryList= []
  for index, row  in dataset.iterrows():
    DictionaryBody = {}
    UniqueBody = []
    [UniqueBody.append(i) for i in row.articleBody if not UniqueBody.count(i)]  

    for bodyWord in UniqueBody:
      DictionaryBody[bodyWord]=0.2
      for headlineWord in row.Headline:
        DictionaryBody[headlineWord]=0.2


    for bodyWord in UniqueBody:
      if bodyWord in row.Headline:
        countWord = row.Headline.count(bodyWord)
        DictionaryBody[bodyWord] = DictionaryBody[bodyWord] + countWord
      else:
        pass
    TotalWords = sum(DictionaryBody.values())
    for key in DictionaryBody:
      DictionaryBody[key] /= TotalWords
    DictionaryList.append(DictionaryBody)
  return DictionaryList

In [0]:
#Get Prbability Distributions for each model
TrainQueryLikelihoodDiscounting         = QueryLikelihoodDiscounting(TrainProcessed)
ValidationQueryLikelihoodDiscounting    = QueryLikelihoodDiscounting(ValidationProcessed)
TestQueryLikelihoodDiscounting          = QueryLikelihoodDiscounting(TestProcessed)

TrainDocumentLikelihoodDiscounting      = DocumentLikelihoodDiscounting(TrainProcessed)
ValidationDocumentLikelihoodDiscounting = DocumentLikelihoodDiscounting(ValidationProcessed)
TestDocumentLikelihoodDiscounting       = DocumentLikelihoodDiscounting(TestProcessed)


# JELINEK-MERCER SMOOTHING

In [0]:
#CREATE FREQUENCY DISTRIBUTION OF ALL WORDS FOR EACH LANGUAGE MODEL! FOR BACKGROUND NOISE

TrainFrequencyHeadline = Counter(chain.from_iterable(TrainProcessed['Headline']))
TrainFrequencyArticleBody = Counter(chain.from_iterable(TrainProcessed['articleBody']))

TestFrequencyHeadline = Counter(chain.from_iterable(TestProcessed['Headline']))
TestFrequencyArticleBody = Counter(chain.from_iterable(TestProcessed['articleBody']))

ValidationFrequencyHeadline = Counter(chain.from_iterable(ValidationProcessed['Headline']))
ValidationFrequencyArticleBody = Counter(chain.from_iterable(ValidationProcessed['articleBody']))

In [0]:
#Function for QueryLieklihood
def QueryLikelihoodJenlick(dataset,DistributionFrequency,tuningParameter):
  DictionaryList= []
  for index, row  in dataset.iterrows():
    DictionaryHeadline = {}
    UniqueHeadline = []
    [UniqueHeadline.append(i) for i in row.Headline if not UniqueHeadline.count(i)]  
    
    for headlineWord in UniqueHeadline:
      DictionaryHeadline[headlineWord]=0 
      for bodyWord in row.articleBody:
        DictionaryHeadline[bodyWord]=0

    for headlineWord in UniqueHeadline:
      countWord = row.articleBody.count(headlineWord)
      firstTerm = tuningParameter*(countWord/len(row.articleBody))
      secondTerm = (1-tuningParameter)*(DistributionFrequency[headlineWord]/sum(DistributionFrequency.values()))
      DictionaryHeadline[headlineWord] = firstTerm + secondTerm
    DictionaryList.append(DictionaryHeadline)
  return DictionaryList

In [0]:
#Function for DocumentLikelihood
def DocumentLikelihoodJenlick(dataset,DistributionFrequency,tuningParameter):
  DictionaryList= []
  for index, row  in dataset.iterrows():
    DictionaryBody = {}
    UniqueBody = []
    [UniqueBody.append(i) for i in row.articleBody if not UniqueBody.count(i)]  

    for bodyWord in UniqueBody:
      DictionaryBody[bodyWord]=0
      for headlineWord in row.Headline:
        DictionaryBody[headlineWord]=0 

    for bodyWord in UniqueBody:
        countWord = row.Headline.count(bodyWord)
        firstTerm = tuningParameter*(countWord/len(row.Headline))
        secondTerm = (1-tuningParameter)*(DistributionFrequency[bodyWord]/sum(DistributionFrequency.values()))
        DictionaryBody[bodyWord] = firstTerm + secondTerm
    DictionaryList.append(DictionaryBody)
  return DictionaryList

In [0]:

#Get Prbability Distributions for each model
TrainQueryLikelihoodJenlick         = QueryLikelihoodJenlick(TrainProcessed,TrainFrequencyArticleBody,i)
ValidationQueryLikelihoodJenlick    = QueryLikelihoodJenlick(ValidationProcessed,ValidationFrequencyArticleBody,i)
TestQueryLikelihoodJenlick          = QueryLikelihoodJenlick(TestProcessed,TestFrequencyArticleBody,i)

TrainDocumentLikelihoodJenlick      = DocumentLikelihoodJenlick(TrainProcessed,TrainFrequencyHeadline,i)
ValidationDocumentLikelihoodJenlick = DocumentLikelihoodJenlick(ValidationProcessed,ValidationFrequencyHeadline,i)
TestDocumentLikelihoodJenlick       = DocumentLikelihoodJenlick(TestProcessed,TestFrequencyHeadline,i)

### HYPERPARAMETER TUNING Jelinek-Mercer

In [0]:
hyper = np.linspace(0.1,1,10)
for i in hyper:
  TrainQueryLikelihoodJenlick         = QueryLikelihoodJenlick(TrainProcessed,TrainFrequencyArticleBody,i)
  ValidationQueryLikelihoodJenlick    = QueryLikelihoodJenlick(ValidationProcessed,ValidationFrequencyArticleBody,i)
  TestQueryLikelihoodJenlick          = QueryLikelihoodJenlick(TestProcessed,TestFrequencyArticleBody,i)

  TrainDocumentLikelihoodJenlick      = DocumentLikelihoodJenlick(TrainProcessed,TrainFrequencyHeadline,i)
  ValidationDocumentLikelihoodJenlick = DocumentLikelihoodJenlick(ValidationProcessed,ValidationFrequencyHeadline,i)
  TestDocumentLikelihoodJenlick       = DocumentLikelihoodJenlick(TestProcessed,TestFrequencyHeadline,i)
  
     ####ENTER CODE TAKEN
  ###FROM PROCESSING AND KL DIVERGENCE BELOW:
  ###### REMOVED TO AVOID CLUTTER

# DIRICHLET SMOOTHING

In [0]:
#CREATE FREQUENCY DISTRIBUTION OF ALL WORDS FOR EACH LANGUAGE MODEL! FOR BACKGROUND NOISE

TrainFrequencyHeadline = Counter(chain.from_iterable(TrainProcessed['Headline']))
TrainFrequencyArticleBody = Counter(chain.from_iterable(TrainProcessed['articleBody']))

TestFrequencyHeadline = Counter(chain.from_iterable(TestProcessed['Headline']))
TestFrequencyArticleBody = Counter(chain.from_iterable(TestProcessed['articleBody']))

ValidationFrequencyHeadline = Counter(chain.from_iterable(ValidationProcessed['Headline']))
ValidationFrequencyArticleBody = Counter(chain.from_iterable(ValidationProcessed['articleBody']))

In [0]:
#Function for QueryLieklihood
def QueryLikelihoodDirchlect(dataset,DistributionFrequency,newtuningParameter):
  DictionaryList= []
  for index, row  in dataset.iterrows():
    DictionaryHeadline = {}
    UniqueHeadline = []
    [UniqueHeadline.append(i) for i in row.Headline if not UniqueHeadline.count(i)]  
    
    for headlineWord in UniqueHeadline:
      DictionaryHeadline[headlineWord]=0 
      for bodyWord in row.articleBody:
        DictionaryHeadline[bodyWord]=0

    for headlineWord in UniqueHeadline:
      articleLength = len(row.articleBody)
      #print(headlineWord+str(articleLength))
      
      countWord = row.articleBody.count(headlineWord)
      a = articleLength/(articleLength+newtuningParameter)
      print(headlineWord +str(a))
      firstTerm = a*countWord/articleLength
      secondTerm = (1-a)*(DistributionFrequency[headlineWord]/sum(DistributionFrequency.values()))
      DictionaryHeadline[headlineWord] = firstTerm + secondTerm
    DictionaryList.append(DictionaryHeadline)
  return DictionaryList

In [0]:
#Function for DocumentLikelihood
def DocumentLikelihoodDirchlect(dataset,DistributionFrequency,newtuningParameter):
  DictionaryList= []
  for index, row  in dataset.iterrows():
    DictionaryBody = {}
    UniqueBody = []
    [UniqueBody.append(i) for i in row.articleBody if not UniqueBody.count(i)]  

    for bodyWord in UniqueBody:
      DictionaryBody[bodyWord]=0
      for headlineWord in row.Headline:
        DictionaryBody[headlineWord]=0 

    for bodyWord in UniqueBody:
        countWord = row.Headline.count(bodyWord)
        a = len(row.Headline)/(len(row.Headline)+newtuningParameter)
        firstTerm = (a)*(countWord/len(row.Headline))
        print(str(bodyWord)+str(a))
        secondTerm = (1-a)*(DistributionFrequency[bodyWord]/sum(DistributionFrequency.values()))
        DictionaryBody[bodyWord] = firstTerm + secondTerm
    DictionaryList.append(DictionaryBody)
  return DictionaryList

In [0]:
#Get Prbability Distributions for each model
TrainQueryLikelihoodDirchlect        = QueryLikelihoodDirchlect(TrainProcessed,TrainFrequencyArticleBody,215)
ValidationQueryLikelihoodDirchlect     = QueryLikelihoodDirchlect(ValidationProcessed,ValidationFrequencyArticleBody,215)
TestQueryLikelihoodDirchlect         = QueryLikelihoodDirchlect(TestProcessed,TestFrequencyArticleBody,215)

TrainDocumentLikelihoodDirchlect      = DocumentLikelihoodDirchlect(TrainProcessed,TrainFrequencyHeadline,10)
ValidationDocumentLikelihoodDirchlect  = DocumentLikelihoodDirchlect(ValidationProcessed,ValidationFrequencyHeadline,10)
TestDocumentLikelihoodDirchlect     = DocumentLikelihoodDirchlect(TestProcessed,TestFrequencyHeadline,10)

### HYPERPARAMETER TUNING DIRICHLET

In [0]:
hyper = np.linspace(100,200,30)
hyper2 = np.linspace(5,10,30)
for i in hyper:
  TrainQueryLikelihoodDirchlect        = QueryLikelihoodDirchlect(TrainProcessed,TrainFrequencyArticleBody,i)
  ValidationQueryLikelihoodDirchlect     = QueryLikelihoodDirchlect(ValidationProcessed,ValidationFrequencyArticleBody,i)
  TestQueryLikelihoodDirchlect         = QueryLikelihoodDirchlect(TestProcessed,TestFrequencyArticleBody,i)

  TrainDocumentLikelihoodDirchlect      = DocumentLikelihoodDirchlect(TrainProcessed,TrainFrequencyHeadline,j)
  ValidationDocumentLikelihoodDirchlect  = DocumentLikelihoodDirchlect(ValidationProcessed,ValidationFrequencyHeadline,j)
  TestDocumentLikelihoodDirchlect     = DocumentLikelihoodDirchlect(TestProcessed,TestFrequencyHeadline,j)
  
    ####ENTER CODE TAKEN
  ###FROM PROCESSING AND KL DIVERGENCE BELOW:
  ###### REMOVED TO AVOID CLUTTER

# PROCESSING AND FINAL KL DIVERGENCE

In [0]:
trainLikelihoods = pd.DataFrame(list(zip(TrainQueryLikelihoodDiscounting, TrainDocumentLikelihoodDiscounting )), columns = ['qldict', 'dldict'])
validationLikelihoods = pd.DataFrame(list(zip(ValidationQueryLikelihoodDiscounting, ValidationDocumentLikelihoodDiscounting )), columns = ['qldict', 'dldict'])
testLikelihoods = pd.DataFrame(list(zip(TestQueryLikelihoodDiscounting, TestDocumentLikelihoodDiscounting )), columns = ['qldict', 'dldict'])

In [0]:
def DictionaryToValues(DictionaryOfProbabilities):
  querylikelihoodsValues = []
  documentlikelihoodsValues = []
  for index,row in DictionaryOfProbabilities.iterrows():
    querylikelihoodtemplist = []
    documentlikelihoodtemplist = []
    for k in row.qldict.keys() & row.dldict.keys():
      querylikelihoodtemplist.append(row.qldict[k])
      documentlikelihoodtemplist.append(row.dldict[k])
    querylikelihoodsValues.append(querylikelihoodtemplist)
    documentlikelihoodsValues.append(documentlikelihoodtemplist)
  return querylikelihoodsValues, documentlikelihoodsValues
    
    
    #print(k, row.qldict[k], row.dldict[k] )
    
    

In [0]:
TrainQueryLikelihoodsValues, TrainDocumentLikelihoodsValues = DictionaryToValues(trainLikelihoods)
ValidationQueryLikelihoodsValues, ValidationDocumentLikelihoodsValues = DictionaryToValues(validationLikelihoods)
TestQueryLikelihoodsValues, TestDocumentLikelihoodsValues = DictionaryToValues(testLikelihoods)

In [0]:
trainLikelihoodValues = pd.DataFrame(list(zip(TrainQueryLikelihoodsValues, TrainDocumentLikelihoodsValues )), columns = ['qlValues', 'dlValues'])
validationLikelihoodValues = pd.DataFrame(list(zip(ValidationQueryLikelihoodsValues, ValidationDocumentLikelihoodsValues )), columns = ['qlValues', 'dlValues'])
testLikelihoods=Values = pd.DataFrame(list(zip(TestQueryLikelihoodsValues, TestDocumentLikelihoodsValues )), columns = ['qlValues', 'dlValues'])

In [0]:
TrainWithLikelihoods = pd.concat([TrainProcessed, trainLikelihoodValues], axis=1, join_axes=[TrainProcessed.index])
ValidationWithLikelihoods = pd.concat([ValidationProcessed, validationLikelihoodValues], axis=1, join_axes=[ValidationProcessed.index])
TestWithLikelihoods = pd.concat([TestProcessed, testLikelihoods], axis=1, join_axes=[TestProcessed.index])

In [0]:
def KL(a, b):
    a = np.asarray(a, dtype=np.float)
    b = np.asarray(b, dtype=np.float)

    return np.sum(np.where(a != 0, a * np.log(a / b), 0))

In [0]:
TrainWithLikelihoods['KLDivergence'] = TrainWithLikelihoods.apply(lambda x: KL(x['qlValues'], x['dlValues']), axis=1)
ValidationWithLikelihoods['KLDivergence'] = ValidationWithLikelihoods.apply(lambda x: KL(x['qlValues'], x['dlValues']), axis=1)
TestWithLikelihoods['KLDivergence'] = TestWithLikelihoods.apply(lambda x: KL(x['qlValues'], x['dlValues']), axis=1)

In [0]:
ValidationWithLikelihoods.groupby('Stance').mean()

In [0]:
TrainWithLikelihoods.groupby('Stance').mean()

In [0]:
ValidationWithLikelihoods.groupby('Stance').mean()
TESTINGPRECISION = TrainWithLikelihoods[TrainWithLikelihoods.Stance != 'unrelated']

In [0]:
groups = TESTINGPRECISION.groupby('Stance')

# Plot
fig, ax = plt.subplots()
 # Optional, just adds 5% padding to the autoscaling
for name, group in groups:
    ax.plot(group.CosineSim, group.KLDivergence, marker='o', linestyle='', ms=4, label=name)
ax.legend()

plt.show()

In [0]:
  #KL DIVERGENCE TESTINGS:
#array1 = ['bog' ,'bag' 'rat']
#array2 = ['hello' , ''word', 'test' ]